### Study area polygons
* Create polygons for Laos, Vietnam, North Vietnam and South Vietnam based on GADM country boundaries which are later used for cropping the final LUC maps
* Create study area polygons defined as intersections between GADM country outlines and topographic map sheet coverage -> statistics on percentage covered by the maps are reported below
* merge input LUC data provided in tiled format for tiles withing the study area

In [1]:
import geopandas as gpd
import numpy as np
import os

from shapely.geometry import box

from config import Config
from core.utils import merge_tifs_within_study_area

In [2]:
config = Config.Config()

In [3]:
for path in config.cropping_geoms.values():
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

In [4]:
ms_index = gpd.read_file(config.map_sheet_index_geo_overedge_path).to_crs(config.output_crs)
ms_area = ms_index.union_all()

## Lao

In [5]:
lao = gpd.read_file(config.lao_gadm_path).to_crs(config.output_crs)
lao.to_file(config.cropping_geoms.get("lao"), driver="GeoJSON")

# Define the study area as intersections of country boundaries with map sheet coverage
study_area_lao = lao.intersection(ms_area)
study_area_lao.to_file(config.study_area_lao_path, driver="GeoJSON")

In [6]:
lao_ms_cover = np.round(study_area_lao.area[0]/lao.geometry.area[0] * 100, 2)
print(f"Map sheets cover {lao_ms_cover}% of total area of Lao PDR (based on GADM boundaries).")

Map sheets cover 98.12% of total area of Lao PDR (based on GADM boundaries).


## Vietnam

In [7]:
vnm = gpd.read_file(config.vnm_gadm_path).to_crs(config.output_crs)

In [8]:
# Create two additional areas of north and south vietnam based on the 17th parallel 
# as the legend between these sheets slightly differs
clipping_gdf_svnm = gpd.GeoDataFrame({"geometry": [box(80, 0, 150, 17)]}, crs=config.map_sheet_index_crs).to_crs(config.output_crs)
clipping_gdf_nvnm = gpd.GeoDataFrame({"geometry": [box(80, 17, 150, 90)]}, crs=config.map_sheet_index_crs).to_crs(config.output_crs)

# Perform the clip operation
svnm = gpd.overlay(vnm, clipping_gdf_svnm, how="intersection")
nvnm = gpd.overlay(vnm, clipping_gdf_nvnm, how="intersection")

# Write out final area geometries
vnm.to_file(config.cropping_geoms.get("vnm"), driver="GeoJSON")
svnm.to_file(config.cropping_geoms.get("svnm"), driver="GeoJSON")
nvnm.to_file(config.cropping_geoms.get("nvnm"), driver="GeoJSON")

In [9]:
# Define the study areas as intersections of country boundaries with map sheet coverage
study_area_vnm = vnm.intersection(ms_area)
study_area_svnm = svnm.intersection(ms_area)
study_area_nvnm = nvnm.intersection(ms_area)

study_area_vnm.to_file(config.study_area_vnm_path, driver="GeoJSON")
study_area_svnm.to_file(config.study_area_svnm_path, driver="GeoJSON")
study_area_nvnm.to_file(config.study_area_nvnm_path, driver="GeoJSON")

In [10]:
vnm_ms_cover = np.round(study_area_vnm.area[0]/vnm.geometry.area[0] * 100, 2)
print(f"Map sheets cover {vnm_ms_cover}% of total area of Vietnam (based on GADM boundaries).")

Map sheets cover 98.92% of total area of Vietnam (based on GADM boundaries).


In [11]:
svnm_ms_cover = np.round(study_area_svnm.area[0]/svnm.geometry.area[0] * 100, 2)
print(f"Map sheets cover {svnm_ms_cover}% of total area of Vietnam south of 17°.")

Map sheets cover 99.73% of total area of Vietnam south of 17°.


In [12]:
nvnm_ms_cover = np.round(study_area_nvnm.area[0]/nvnm.geometry.area[0] * 100, 2)
print(f"Map sheets cover {nvnm_ms_cover}% of total area of Vietnam north of 17°.")

Map sheets cover 98.31% of total area of Vietnam north of 17°.


## Vietnam and Laos combined

In [13]:
study_area= study_area_vnm.union(study_area_lao)
study_area.to_file(config.study_area_path, driver="GeoJSON")

# Merge tiled external LUC data 

In [14]:
# Study area
study_area = gpd.read_file(config.study_area_path).to_crs("EPSG:4326")

# Mangrove study area (EEZ land union polygon)
eez = gpd.read_file(config.vnm_eez_land_path)
vnm_eez = eez[eez["UNION"] == "Vietnam"].reset_index(drop=True).to_crs("EPSG:4326")

In [15]:
# Global Mangrove Watch
merge_tifs_within_study_area(
    input_folder=config.gmw_1996_folder,
    output_file=config.gmw_1996_merged_path,
    study_area=vnm_eez
)

Merged TIFF saved as ../data/raw/luc/gmw_1996_merged.tif


In [16]:
# GLC_FCS30D 1985
merge_tifs_within_study_area(
    input_folder=config.luc_fcs30_folder,
    output_file=config.luc_fcs30_1985_path,
    study_area=study_area,
    file_prefix="GLC_FCS30_19852000",
    indexes=[1]
)

Merged TIFF saved as ../data/raw/luc/luc_glc_fcs30_1985.tif


In [17]:
# GLC_FCS30D 1990
merge_tifs_within_study_area(
    input_folder=config.luc_fcs30_folder,
    output_file=config.luc_fcs30_1990_path,
    study_area=study_area,
    file_prefix="GLC_FCS30_19852000",
    indexes=[2]
)

Merged TIFF saved as ../data/raw/luc/luc_glc_fcs30_1990.tif
